In [ ]:
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage
!pip install sodapy

In [3]:
#import necessary libraries
import pandas as pd
import numpy as np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO

s1 --> Gather --> Decompress --> Convert to Tabular --> Clean --> Reformat --> Consolidate --> Tranformation --> Load

In [7]:
from sodapy import Socrata
import pandas as pd
from sodapy import Socrata

data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='vww9-qguh'    # The data set at the API endpoint (311 data in this case)
app_token='m93ORKHDVAEgZJ48OfuDNs5sI'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint
# Set the timeout to 60 seconds
client.timeout = 60
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)
# Save the data frame to a CSV file

In [4]:
# Function

import os
import boto3
from azure.storage.blob import BlobServiceClient
from google.cloud import storage
import pandas as pd
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [11]:
from sodapy import Socrata
import pandas as pd
from sodapy import Socrata

data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='vww9-qguh'    # The data set at the API endpoint (311 data in this case)
app_token='m93ORKHDVAEgZJ48OfuDNs5sI'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint
# Set the timeout to 60 seconds
client.timeout = 60
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set, limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df_raw = pd.DataFrame.from_records(results)
df_raw.dropna()
df_raw.shape
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   dbn                         2000 non-null   object
 1   school_name                 2000 non-null   object
 2   grade                       2000 non-null   object
 3   year                        2000 non-null   object
 4   demographic_category        2000 non-null   object
 5   demographic_variable        2000 non-null   object
 6   total_days                  2000 non-null   object
 7   days_absent                 2000 non-null   object
 8   days_present                2000 non-null   object
 9   attendance                  2000 non-null   object
 10  contributing_20_total_days  2000 non-null   object
 11  chronically_absent          2000 non-null   object
 12  chronically_absent_1        2000 non-null   object
dtypes: object(13)
memory usage: 203.2+ KB


In [9]:
df_clean = df_raw.copy()

In [17]:
df_clean = df_raw.drop(columns = ['ehail_fee'])
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56551 entries, 0 to 56550
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               56551 non-null  int32         
 1   lpep_pickup_datetime   56551 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  56551 non-null  datetime64[ns]
 3   store_and_fwd_flag     53136 non-null  object        
 4   RatecodeID             53136 non-null  float64       
 5   PULocationID           56551 non-null  int32         
 6   DOLocationID           56551 non-null  int32         
 7   passenger_count        53136 non-null  float64       
 8   trip_distance          56551 non-null  float64       
 9   fare_amount            56551 non-null  float64       
 10  extra                  56551 non-null  float64       
 11  mta_tax                56551 non-null  float64       
 12  tip_amount             56551 non-null  float64       
 13  t

Create Connection to Azure Container
- connection string

In [ ]:
#specify the path to your JSON configuration file
config_file_path = 'config.json'

#load the JSON configuration file
with open(config_file_path, 'r') as config_file
    config = json.load(config_file)

#print the configuration
#Connection_STRING = config['CONNECTION_STRING_AZURE_STORAGE']

CONNECTION_STRING_AZURE_STORAGE = config['connectionString']
CONTAINER_AZURE = 'studentattendance'
blob_name = 'nycatt.csv'

# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

In [ ]:
# Example usage
if __name__ == "__main__":

    output = StringIO()
    df_raw.to_csv(output, index=False)
    data = output.getvalue()
    output.close()

    # Azure
    azure_connect_str = 'YOUR_AZURE_STORAGE_CONNECTION_STRING'
    azure_container_name = 'YOUR_CONTAINER_NAME'
    azure_blob_name = 'your_blob_name.csv'
    azure_data = 'Hello, Azure!'
    azure_upload_blob(azure_connect_str, azure_container_name, azure_blob_name, azure_data)
    # Download and print the blob content
    azure_blob_content = azure_download_blob(azure_connect_str, azure_container_name, azure_blob_name)
    print(azure_blob_content)